# Genarate Graphs and Labels for training

## Module Import

In [1]:
'''

+-------------------------------+
|                               |
|    I M P O R T   M O D E L    | 
|                               |
+-------------------------------+

Description

Updated on Thu Aug 4 11:12 2022

@author: Xingyu Wu
'''

# --------------------------
# Importing public libraries
# --------------------------

# Operating system specific functions
import os

# Argument parser, for configuring the program execution
import argparse

# An object oriented library for handling EPANET files in Python
import epynet 

# yaml / yml configuration file support (a kind of language) 
# pip install pyyaml
import yaml

# PyTorch deep learning framework
import torch

# Import the networkx library
import networkx as nx

# Import Pandas for data handling
import pandas as pd

# Import numpy for array handling
import numpy as np

# Matplotlib for generating graphics
import matplotlib.pyplot as plt

# PyTorch from graph conversion tool
from torch_geometric.utils import from_networkx

# conda install pytorch-sparse -c pyg

# Train-test split with shuffle 
from sklearn.model_selection import train_test_split

/home/zczlx79/.conda/envs/pytorch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Modules

In [2]:
# --------------------------
# Importing custom libraries
# --------------------------

# Import a custom tool for converting EPANET .inp files to networkx graphs
from utils_pre.epanet_loader import get_nx_graph

# Function for visualisationa
from utils_pre.visualisation import visualise

# EPANET simulator, used to generate nodal pressures from the nominal model
from utils_pre.epanet_simulator import epanetSimulator

# SCADA timeseries dataloader
from utils_pre.data_loader import battledimLoader, dataCleaner, dataGenerator

# PyTorch early stopping callback
from utils_pre.early_stopping import EarlyStopping

# Metrics
from utils_pre.metrics import Metrics


# 1. Process.py Notes

## 1.1 Preparation

### Importing public libraries

In [3]:
import pandas as pd
from datetime import datetime
from scipy.sparse import csr_matrix
import pickle as pkl

### Environment Paths

In [3]:
path_to_wdn = './data/L-TOWN.inp' # Do I need to distinguish between REAL and NOMINAL EPANET inps here? 

### `EPANET` Model Import

In [4]:
# Import the .inp file using the EPYNET library
wdn = epynet.Network(path_to_wdn)

# Solve hydraulic model for a single timestep
wdn.solve()

### `networkx` Graph Conversion

In [5]:
# Convert the file using a custom function, based on:
# https://github.com/BME-SmartLab/GraphConvWat 
# G: Graph in nx format; pos: node position; head: hydraulic heads which not used in this project
G , pos , head = get_nx_graph(wdn, weight_mode='pipe_length', get_head=True)

### Configuration File Import-->find the nodes with pressure sensor

In [3]:
'''
"dataset_configuration.yalm" :
Configuration file for generating the Historical (2018) and Evaluation (2019) datasets of the BattLeDIM competition.
Contains:
-file name of network
-dataset start and end times
-leakage information (link ID, start Time, end Time, leak Diameter (m), leak Type, peak Time)
-sensor locations (link or node IDs)
'''
print('Importing dataset configuration...\n')

# Open the dataset configuration file
with open('./data/dataset_configuration.yaml') as file:

    # Load the configuration to a dictionary
    config = yaml.load(file, Loader=yaml.FullLoader)

# Generate a list of integers, indicating the number of the node
# at which a  pressure sensor is present
sensors = [int(string.replace("n", "")) for string in config['pressure_sensors']]

Importing dataset configuration...



### Visualize pressure sensors

In [ ]:
# print('Pressure sensors in the network are located at nodes: ')
# for each in sensors:
#     print('Node: \t {}'.format(each))

## 1.2 from SCADA Dataset get Pressure Dataframe

### The number of WDN Graphs edges and nodes

In [4]:
# len(G.edges) # 905

In [6]:
# G.nodes # 1-782

### Input: the 2018 SCADA Pressures

In [13]:
df_pressure = pd.read_csv('./data/2018_SCADA_Pressures.csv', sep=';', decimal=',')

### Ued batteldimLoader from utils_pre.data_loader to fill in the values of non-observed points

In [14]:
# Load the data into a numpy array with format matching the GraphConvWat problem
pressure_2018 = battledimLoader(observed_nodes = sensors,
                                n_nodes        = 782,
                                path           = './data/',
                                file           = '2018_SCADA_Pressures.csv')

# Print information and instructions about the imported data
msg = "The imported sensor data has shape (i,n,d): {}".format(pressure_2018.shape)

print(msg + "\n" + len(msg)*"-" + "\n")
print("Where: ")
print("'i' is the number of observations: {}".format(pressure_2018.shape[0]))
print("'n' is the number of nodes: {}".format(pressure_2018.shape[1]))
print("'d' is a {}-dimensional vector consisting of the pressure value and a mask ".format(pressure_2018.shape[2]))
print("The mask is set to '1' on observed nodes and '0' otherwise\n")

print("\n" + len(msg)*"-" + "\n")

The imported sensor data has shape (i,n,d): (105120, 782, 2)
------------------------------------------------------------

Where: 
'i' is the number of observations: 105120
'n' is the number of nodes: 782
'd' is a 2-dimensional vector consisting of the pressure value and a mask 
The mask is set to '1' on observed nodes and '0' otherwise


------------------------------------------------------------



### df_pressure

In [16]:
df_pressure_new = pd.DataFrame(pressure_2018[:,:,0])

In [17]:
df_pressure_new['datetime'] = df_pressure['Timestamp']

In [18]:
df_pressure_new

,0,1,2,3,4,5,6,7,8,9,...,773,774,775,776,777,778,779,780,781,datetime
0,28.92,0.0,0.0,33.87,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-01-01 00:00:00
1,28.94,0.0,0.0,33.89,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-01-01 00:05:00
2,28.96,0.0,0.0,33.91,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-01-01 00:10:00
3,28.98,0.0,0.0,33.93,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-01-01 00:15:00
4,28.98,0.0,0.0,33.93,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-01-01 00:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,28.55,0.0,0.0,33.61,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-12-31 23:35:00
105116,28.56,0.0,0.0,33.61,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-12-31 23:40:00
105117,28.58,0.0,0.0,33.64,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-12-31 23:45:00
105118,28.58,0.0,0.0,33.64,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-12-31 23:50:00


In [19]:
df_pressure_new.iloc[287:289]

,0,1,2,3,4,5,6,7,8,9,...,773,774,775,776,777,778,779,780,781,datetime
287,28.81,0.0,0.0,33.75,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-01-01 23:55:00
288,28.83,0.0,0.0,33.77,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-01-02 00:00:00


### Generate graphs for training (only 2018 dataset used as training dataset)

In [20]:
begin = datetime.date(2018,1,1)
end = datetime.date(2018,12,31)
graphs = []
for i in range((end - begin).days+1): # range(365)
    tmp_feature = []
    for node in G.nodes(): # 1-782
        tmp_feature.append(np.array(df_pressure_new.iloc[288*i : 288*(i+1), node-1].tolist()))
    G.graph["feature"] = csr_matrix(tmp_feature) 
    graphs.append(G.copy())

In [21]:
len(graphs)

365

In [22]:
graphs[364].graph["feature"] # 9504/33 = 288 = (60/5) * 24

<782x288 sparse matrix of type '<class 'numpy.float64'>'
	with 9504 stored elements in Compressed Sparse Row format>

In [23]:
graphs[364][780][781] # adjs[364][779, 780]

{'weight': 0.5911166229382008, 'name': 'p901'}

In [24]:
# save_path = './data/graphs/graph.pkl'
# with open(save_path, "wb") as f:
#     pkl.dump(graphs, f)
print("Processed Data Saved at {}".format(save_path))

Processed Data Saved at ./data/graphs/graph.pkl


In [4]:
# save_path = './data/graphs/graph_2018_unweighted.pkl'
# with open(save_path, "rb") as f:
#     graphs_load = pkl.load(f)
# print("Loaded {} graphs".format(len(graphs_load)))

Loaded 365 graphs


# 2. Leakage Label for training

## 2.1 Load the df_labels and overview

In [ ]:
label_dir = './data/2018_Leakages.csv'
df_label = pd.read_csv(label_dir, sep=';', decimal=',')
df_label

,Timestamp,p31,p158,p183,p232,p257,p369,p427,p461,p538,p628,p654,p673,p810,p866
0,2018-01-01 00:00:00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0
1,2018-01-01 00:05:00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0
2,2018-01-01 00:10:00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0
3,2018-01-01 00:15:00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0
4,2018-01-01 00:20:00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,2018-12-31 23:35:00,0.0,0.0,0.0,0.0,6.78,0.0,5.05,0.0,0.0,0.0,5.44,0.0,6.86,0.0
105116,2018-12-31 23:40:00,0.0,0.0,0.0,0.0,6.78,0.0,5.04,0.0,0.0,0.0,5.44,0.0,6.86,0.0
105117,2018-12-31 23:45:00,0.0,0.0,0.0,0.0,6.79,0.0,5.05,0.0,0.0,0.0,5.45,0.0,6.86,0.0
105118,2018-12-31 23:50:00,0.0,0.0,0.0,0.0,6.79,0.0,5.05,0.0,0.0,0.0,5.44,0.0,6.86,0.0


## 2.2 Visualize Leakage pipes and corresponding nodes

In [ ]:
leakage_list = df_label.columns.tolist()
for node in graphs[0]: # 782个节点
    for nbr in sorted(graphs[0].neighbors(node)): # 每个节点对应的邻居节点
        if graphs[0][node][nbr]['name'] in leakage_list:
            leakage_list.remove(graphs[0][node][nbr]['name'])
            print("leakage pipe {}'s nodes are: {} and {}.".format(graphs[i][node][nbr]['name'], node, nbr))

leakage pipe p31's nodes are: 40 and 42.
leakage pipe p461's nodes are: 106 and 484.
leakage pipe p538's nodes are: 147 and 531.
leakage pipe p232's nodes are: 184 and 191.
leakage pipe p628's nodes are: 197 and 590.
leakage pipe p673's nodes are: 206 and 623.
leakage pipe p158's nodes are: 232 and 235.
leakage pipe p183's nodes are: 299 and 305.
leakage pipe p866's nodes are: 311 and 758.
leakage pipe p427's nodes are: 340 and 462.
leakage pipe p257's nodes are: 350 and 351.
leakage pipe p369's nodes are: 422 and 423.
leakage pipe p654's nodes are: 609 and 610.
leakage pipe p810's nodes are: 716 and 717.


## 2.3 Generate labels for training

In [ ]:
begin = datetime.date(2018,1,1)
end = datetime.date(2018,12,31)
label = np.zeros((365, 782))
for i in range((end - begin).days+1): # range(365)
    # print('-'*40)
    # print("This is the No.{} day".format(i+1))
    # print('-'*40)
    for node in graphs[i]: # 当前时间步里的782个节点 1-782
        for nbr in sorted(graphs[i].neighbors(node)): # 每个节点对应的邻居节点 1-782
            if graphs[i][node][nbr]['name'] in df_label.columns.tolist(): # 如果这两个节点构成的边属性'name'在泄露管道列表中
                # print("leakage pipe {}'s nodes are: {} and {}.".format(graphs[i][node][nbr]['name'], node, nbr))
                if df_label[graphs[i][node][nbr]['name']][288*i : 288*(i+1)].max() > 0:
                    label[i, node-1] = 1 # 注意node索引-1
                    label[i, nbr-1] = 1

## 2.4 Draft and Validation

In [ ]:
# 求某个节点的邻居节点
for nbr in sorted(graphs[0].neighbors(782)):
    print(nbr)

341
342


In [ ]:
# 对于处理好的[365, 782]label矩阵进行1类别的计数
count = 0
for i in range(label.shape[0]):
    for j in range(label.shape[1]):
        if label[i, j] == 1:
            count += 1
count

2536

In [ ]:
print(np.count_nonzero(label == 1))

2536


In [ ]:
# 对于df_label，在365天里所有管道的泄露天数和，对上一单元格结果进行验证(是2536的一半证明正确)
leakage_list = df_label.columns.tolist()
leakage_list.remove('Timestamp')
for i in range((end - begin).days+1): # range(365)
    for leak_pipe in leakage_list:
        if df_label[leak_pipe][288*i : 288*(i+1)].max() > 0:
            count += 1
count

1268

# 1. Generating training input with generated dataset.

## 1.1 Preparation

### Importing public libraries

In [4]:
import pandas as pd
from datetime import datetime
from scipy.sparse import csr_matrix
import pickle as pkl
import sys

### Environment Paths

In [5]:
path_to_wdn = './data/L-TOWN.inp' # Do I need to distinguish between REAL and NOMINAL EPANET inps here? 

### `EPANET` Model Import

In [6]:
# Import the .inp file using the EPYNET library
wdn = epynet.Network(path_to_wdn)

# Solve hydraulic model for a single timestep
wdn.solve()

### `networkx` Graph Conversion

In [7]:
# Convert the file using a custom function, based on:
# https://github.com/BME-SmartLab/GraphConvWat 
# G: Graph in nx format; pos: node position; head: hydraulic heads which not used in this project
G , pos , head = get_nx_graph(wdn, weight_mode='pipe_length', get_head=True)

### Configuration File Import-->find the nodes with pressure sensor

In [8]:
'''
"dataset_configuration.yalm" :
Configuration file for generating the Historical (2018) and Evaluation (2019) datasets of the BattLeDIM competition.
Contains:
-file name of network
-dataset start and end times
-leakage information (link ID, start Time, end Time, leak Diameter (m), leak Type, peak Time)
-sensor locations (link or node IDs)
'''
print('Importing dataset configuration...\n')

# Open the dataset configuration file
with open('./data/dataset_configuration.yaml') as file:

    # Load the configuration to a dictionary
    config = yaml.load(file, Loader=yaml.FullLoader)

# Generate a list of integers, indicating the number of the node
# at which a  pressure sensor is present
sensors = [int(string.replace("n", "")) for string in config['pressure_sensors']]

Importing dataset configuration...



## 1.2 from SCADA Dataset get Pressure Dataframe

### !!! Customize

In [9]:
year_start_ = 2016
year_end_ = 2017
mode = 'real'
# mode = 'nominal'

### The number of WDN Graphs edges and nodes

In [10]:
# len(G.edges) # 905

In [11]:
# G.nodes # 1-782

### 1.2.1 Obtain SCADA Pressure csv file

#### Input: generated measurements

In [12]:
measure_path = './data/generator_data/{}_{}_{}/Measurements.xlsx'.format(year_start_, year_end_, mode)
df_pressure = pd.read_excel(measure_path, sheet_name='Pressures (m)')

In [13]:
df_pressure # 365 x 2 x 288 = 210240 + 288 = 210528 rows since 2016 has 366 days

,Timestamp,n1,n4,n31,n54,n105,n114,n163,n188,n215,...,n549,n613,n636,n644,n679,n722,n726,n740,n752,n769
0,2016-01-01 00:00:00,28.92,33.87,37.13,37.13,50.52,53.99,52.56,55.32,39.09,...,54.80,56.16,45.53,47.62,47.33,46.14,47.12,43.81,49.17,48.50
1,2016-01-01 00:05:00,28.94,33.89,37.16,37.23,50.60,54.09,52.67,55.42,39.10,...,54.90,56.26,45.61,47.71,47.39,46.19,47.17,43.83,49.22,48.54
2,2016-01-01 00:10:00,28.96,33.91,37.18,37.21,50.57,54.05,52.64,55.39,39.09,...,54.87,56.24,45.61,47.71,47.41,46.20,47.19,43.84,49.24,48.56
3,2016-01-01 00:15:00,28.98,33.93,37.19,37.27,50.60,54.08,52.70,55.45,39.09,...,54.92,56.30,45.65,47.75,47.43,46.21,47.21,43.84,49.26,48.57
4,2016-01-01 00:20:00,28.98,33.93,37.19,37.14,50.53,54.00,52.57,55.32,39.09,...,54.80,56.17,45.54,47.64,47.34,46.15,47.13,43.81,49.17,48.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210523,2017-12-31 23:35:00,28.54,33.48,36.79,34.92,49.06,52.20,50.47,53.24,39.03,...,52.88,54.00,43.59,45.56,45.58,44.82,45.34,43.16,47.51,47.23
210524,2017-12-31 23:40:00,28.55,33.49,36.80,34.82,49.04,52.18,50.38,53.16,39.03,...,52.81,53.91,43.52,45.51,45.55,44.83,45.34,43.16,47.50,47.22
210525,2017-12-31 23:45:00,28.56,33.49,36.81,34.95,49.08,52.21,50.50,53.27,39.04,...,52.90,54.03,43.61,45.58,45.59,44.84,45.36,43.16,47.53,47.24
210526,2017-12-31 23:50:00,28.58,33.51,36.83,34.96,49.13,52.28,50.53,53.29,39.03,...,52.95,54.05,43.62,45.58,45.59,44.84,45.35,43.17,47.52,47.24


#### Delete no.366 day in every year

In [14]:
df_pressure['Timestamp'][105119].dayofyear

365

In [15]:
df_pressure['Timestamp'][105120].dayofyear

366

In [16]:
# 删除第366天
condition = df_pressure['Timestamp'].map(lambda x: x.dayofyear) != 366
df_pressure_ = df_pressure[condition]
df_pressure_ = df_pressure_.reset_index(drop=True)

In [17]:
df_pressure_ # 365 x 2 x 288 = 210240

,Timestamp,n1,n4,n31,n54,n105,n114,n163,n188,n215,...,n549,n613,n636,n644,n679,n722,n726,n740,n752,n769
0,2016-01-01 00:00:00,28.92,33.87,37.13,37.13,50.52,53.99,52.56,55.32,39.09,...,54.80,56.16,45.53,47.62,47.33,46.14,47.12,43.81,49.17,48.50
1,2016-01-01 00:05:00,28.94,33.89,37.16,37.23,50.60,54.09,52.67,55.42,39.10,...,54.90,56.26,45.61,47.71,47.39,46.19,47.17,43.83,49.22,48.54
2,2016-01-01 00:10:00,28.96,33.91,37.18,37.21,50.57,54.05,52.64,55.39,39.09,...,54.87,56.24,45.61,47.71,47.41,46.20,47.19,43.84,49.24,48.56
3,2016-01-01 00:15:00,28.98,33.93,37.19,37.27,50.60,54.08,52.70,55.45,39.09,...,54.92,56.30,45.65,47.75,47.43,46.21,47.21,43.84,49.26,48.57
4,2016-01-01 00:20:00,28.98,33.93,37.19,37.14,50.53,54.00,52.57,55.32,39.09,...,54.80,56.17,45.54,47.64,47.34,46.15,47.13,43.81,49.17,48.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210235,2017-12-31 23:35:00,28.54,33.48,36.79,34.92,49.06,52.20,50.47,53.24,39.03,...,52.88,54.00,43.59,45.56,45.58,44.82,45.34,43.16,47.51,47.23
210236,2017-12-31 23:40:00,28.55,33.49,36.80,34.82,49.04,52.18,50.38,53.16,39.03,...,52.81,53.91,43.52,45.51,45.55,44.83,45.34,43.16,47.50,47.22
210237,2017-12-31 23:45:00,28.56,33.49,36.81,34.95,49.08,52.21,50.50,53.27,39.04,...,52.90,54.03,43.61,45.58,45.59,44.84,45.36,43.16,47.53,47.24
210238,2017-12-31 23:50:00,28.58,33.51,36.83,34.96,49.13,52.28,50.53,53.29,39.03,...,52.95,54.05,43.62,45.58,45.59,44.84,45.35,43.17,47.52,47.24


In [18]:
## check
# df_pressure_.iloc[105119] # 2016-12-30 23:55:00 的数据
# df_pressure_.iloc[105120] # 2017-01-01 00:00:00 的数据

#### Save as the SCADA_Pressure csv file

In [19]:
df_pressure_.to_csv('./data/generator_data/{}_{}_{}/{}_{}_SCADA_Pressures_{}.csv'.format(year_start_, year_end_, mode, year_start_, year_end_, mode), sep=';', decimal=',', index = False)

### 1.2.2 Load the pressure and Fill in the values of non-observed points

In [20]:
path_ = './data/generator_data/{}_{}_{}/'.format(year_start_, year_end_, mode)
file_ = '{}_{}_SCADA_Pressures_{}.csv'.format(year_start_, year_end_, mode)

In [21]:
# Load the data into a numpy array with format matching the GraphConvWat problem
pressure = battledimLoader(observed_nodes = sensors,
                                n_nodes        = 782,
                                path           = path_,
                                file           = file_)

# Print information and instructions about the imported data
msg = "The imported sensor data has shape (i,n,d): {}".format(pressure.shape)

print(msg + "\n" + len(msg)*"-" + "\n")
print("Where: ")
print("'i' is the number of observations: {}".format(pressure.shape[0]))
print("'n' is the number of nodes: {}".format(pressure.shape[1]))
print("'d' is a {}-dimensional vector consisting of the pressure value and a mask ".format(pressure.shape[2]))
print("The mask is set to '1' on observed nodes and '0' otherwise\n")

print("\n" + len(msg)*"-" + "\n")

The imported sensor data has shape (i,n,d): (210240, 782, 2)
------------------------------------------------------------

Where: 
'i' is the number of observations: 210240
'n' is the number of nodes: 782
'd' is a 2-dimensional vector consisting of the pressure value and a mask 
The mask is set to '1' on observed nodes and '0' otherwise


------------------------------------------------------------



In [22]:
df_pressure_new = pd.DataFrame(pressure[:,:,0])

In [23]:
df_pressure_new['datetime'] = df_pressure_['Timestamp']

In [24]:
df_pressure_new

,0,1,2,3,4,5,6,7,8,9,...,773,774,775,776,777,778,779,780,781,datetime
0,28.92,0.0,0.0,33.87,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-01 00:00:00
1,28.94,0.0,0.0,33.89,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-01 00:05:00
2,28.96,0.0,0.0,33.91,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-01 00:10:00
3,28.98,0.0,0.0,33.93,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-01 00:15:00
4,28.98,0.0,0.0,33.93,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-01 00:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210235,28.54,0.0,0.0,33.48,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-12-31 23:35:00
210236,28.55,0.0,0.0,33.49,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-12-31 23:40:00
210237,28.56,0.0,0.0,33.49,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-12-31 23:45:00
210238,28.58,0.0,0.0,33.51,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-12-31 23:50:00


## 1.3 Generate graphs for training

In [25]:
begin_year = df_pressure_new['datetime'][0].date().year
end_year = df_pressure_new['datetime'].iloc[-1].date().year
graphs = []
for i in range((end_year - begin_year + 1)*365): # range(365)
    tmp_feature = []
    for node in G.nodes(): # 1-782
        tmp_feature.append(np.array(df_pressure_new.iloc[288*i : 288*(i+1), node-1].tolist()))
    G.graph["feature"] = csr_matrix(tmp_feature) 
    graphs.append(G.copy())

#### Verification

In [26]:
len(graphs)

730

In [27]:
graphs[729].graph["feature"] # 9504/33 = 288 = (60/5) * 24

<782x288 sparse matrix of type '<class 'numpy.float64'>'
	with 9504 stored elements in Compressed Sparse Row format>

In [28]:
graphs[729][780][781] # adjs[729][779, 780]

{'weight': 0.5911166229382008, 'name': 'p901'}

#### Save the Graphs

In [29]:
path_ = './data/generator_data/{}_{}_{}/'.format(year_start_, year_end_, mode)
save_path = path_ + 'graph_{}_{}_{}.pkl'.format(year_start_, year_end_, mode)
with open(save_path, "wb") as f:
    pkl.dump(graphs, f)
print("Processed Data Saved at {}".format(save_path))

Processed Data Saved at ./data/generator_data/2016_2017_real/graph_2016_2017_real.pkl


#### Load the Graphs

In [30]:
path_ = './data/generator_data/{}_{}_{}/'.format(year_start_, year_end_, mode)
save_path = path_ + 'graph_{}_{}_{}.pkl'.format(year_start_, year_end_, mode)
with open(save_path, "rb") as f:
    graphs = pkl.load(f)
print("Loaded {} graphs".format(len(graphs)))

Loaded 730 graphs


# 2. Leakage Label for training

## 2.1 Load the configuration file

In [33]:
path_ = './data/generator_data/{}_{}_{}/'.format(year_start_, year_end_, mode)
config_path = path_ + 'dataset_configuration_{}_{}.yaml'.format(year_start_, year_end_)
try:
    with open(config_path,'r') as f:
        print('f', f)
        leak_pipes = yaml.load(f.read(),Loader=yaml.FullLoader)
except:
    print('"dataset_configuration" file not found.')
    sys.exit()

f <_io.TextIOWrapper name='./data/generator_data/2016_2017_real/dataset_configuration_2016_2017.yalm' mode='r' encoding='UTF-8'>


In [34]:
start_time = leak_pipes['times']['StartTime']
end_time = leak_pipes['times']['EndTime']
start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M')
end_time = datetime.strptime(end_time, '%Y-%m-%d %H:%M')
leakages = leak_pipes['leakages']
leakages = leakages[1:]
number_of_leaks = len(leakages)

## 2.2 Transfer leakage info to dataframe

In [35]:
df_leakage = pd.DataFrame(leakages)
column_names = {0:'linkID', 1:'startTime', 2:'endTime', 3:'leakDiameter (m)', 4:'leakType', 5:'peakTime'}
df_leakage = pd.concat([df_leakage.iloc[:, 0].str.split(', ', expand=True)], axis=1)
df_leakage.rename(columns = column_names, inplace = True)
df_leakage

,linkID,startTime,endTime,leakDiameter (m),leakType,peakTime
0,p157,2016-01-08 13:30,2017-12-31 23:55,0.011843,incipient,2016-01-25 08:30
1,p361,2016-01-23 04:25,2016-04-02 11:40,0.021320,incipient,2016-03-27 20:35
2,p332,2016-01-31 02:35,2016-02-10 09:20,0.020108,incipient,2016-02-03 16:05
3,p27,2016-02-13 08:25,2017-12-31 23:55,0.0090731,incipient,2016-05-14 19:25
4,p473,2016-03-05 15:45,2016-03-23 10:25,0.022916,abrupt,2016-03-05 15:45
5,p610,2016-07-28 03:05,2017-12-31 23:55,0.010028,incipient,2016-11-02 22:25
6,p828,2016-05-02 14:55,2016-05-29 21:20,0.022318,incipient,2016-05-16 08:00
7,p638,2016-05-18 08:35,2016-06-02 06:05,0.021731,abrupt,2016-05-18 08:35
8,p566,2016-06-01 09:05,2016-06-12 03:00,0.018108,abrupt,2016-06-01 09:05
9,p231,2016-06-28 10:35,2016-08-12 17:30,0.016389,incipient,2016-08-03 02:45


## 2.3 Visualize leakage pipes and corresponding nodes

In [36]:
leakage_list = df_leakage['linkID'].to_list()
for node in graphs[0]: # 782个节点
    for nbr in sorted(graphs[0].neighbors(node)): # 每个节点对应的邻居节点
        if graphs[0][node][nbr]['name'] in leakage_list:
            leakage_list.remove(graphs[0][node][nbr]['name'])
            print("leakage pipe {}'s nodes are: {} and {}.".format(graphs[0][node][nbr]['name'], node, nbr)) # 33 leakage pipes in 2016-2017

leakage pipe p253's nodes are: 1 and 347.
leakage pipe p242's nodes are: 8 and 11.
leakage pipe p27's nodes are: 35 and 38.
leakage pipe p332's nodes are: 75 and 399.
leakage pipe p393's nodes are: 76 and 439.
leakage pipe p231's nodes are: 80 and 144.
leakage pipe p479's nodes are: 115 and 495.
leakage pipe p473's nodes are: 120 and 491.
leakage pipe p521's nodes are: 130 and 522.
leakage pipe p527's nodes are: 138 and 525.
leakage pipe p531's nodes are: 152 and 527.
leakage pipe p566's nodes are: 165 and 548.
leakage pipe p610's nodes are: 182 and 579.
leakage pipe p612's nodes are: 186 and 580.
leakage pipe p683's nodes are: 218 and 628.
leakage pipe p686's nodes are: 219 and 630.
leakage pipe p154's nodes are: 223 and 224.
leakage pipe p155's nodes are: 226 and 227.
leakage pipe p726's nodes are: 230 and 659.
leakage pipe p157's nodes are: 231 and 234.
leakage pipe p158's nodes are: 232 and 235.
leakage pipe p714's nodes are: 235 and 651.
leakage pipe p169's nodes are: 269 and 273.

## 2.4 Generate labels for training

In [37]:
begin_year = start_time.year
end_year = end_time.year
days_num = (end_year - begin_year + 1)*365 # 2*365
label_binary = np.zeros((days_num, 782)) # 0:health; 1:leakage
label_multi = np.zeros((days_num, 782))  # 0:health; 1:incipient; 2:abrupt
leakage_list = df_leakage['linkID'].to_list()
for i in range(days_num): # range(2*365) 遍历两年内的每一天
    # print('-'*40)
    # print("This is the No.{} day".format(i+1))
    # print('-'*40)
    year_i = begin_year + (i+1) // 365 # 整除作为年 
    day_i = (i+1) % 365 # 余数作为天数
    if day_i == 0:
        day_i = 365
        year_i -= 1 
    for node in graphs[i]: # 当前时间步里的782个节点 1-782
        for nbr in sorted(graphs[i].neighbors(node)): # 每个节点对应的邻居节点 1-782
            if graphs[i][node][nbr]['name'] in leakage_list: # 如果这两个节点构成的边属性'name'在泄露管道列表中
                # print("leakage pipe {}'s nodes are: {} and {}.".format(graphs[i][node][nbr]['name'], node, nbr))
                linkID = graphs[i][node][nbr]['name']
                leakage_info = df_leakage[df_leakage['linkID'] == linkID]
                start_date_str, end_date_str = leakage_info['startTime'].values[0], leakage_info['endTime'].values[0]
                start_date, end_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M'), datetime.strptime(end_date_str, '%Y-%m-%d %H:%M')
                start_date_year, end_date_year = start_date.year, end_date.year
                start_date_day, end_date_day = int(start_date.date().strftime('%j')), int(end_date.date().strftime('%j'))
                # 如果year_i与此泄露开始时间在同一年，比较天数看是否处于泄露状态;如果year_i大于泄漏开始年份且<=终止年份，则处于泄漏状态
                if  (year_i == start_date_year and start_date_day <= day_i <= end_date_day) or start_date_year < year_i <= end_date_year: 
                    label_binary[i, node-1] = 1 # 注意node索引-1
                    label_binary[i, nbr-1] = 1
                    if leakage_info['leakType'].item() == 'abrupt':
                        label_multi[i, node-1] = 2 # 注意node索引-1
                        label_multi[i, nbr-1] = 2
                    elif leakage_info['leakType'].item() == 'incipient':
                        cur_node, cur_nbr = label_multi[i, node-1], label_multi[i, nbr-1]
                        label_multi[i, node-1] = max(1, cur_node) # 优先标记为 2 abrupt
                        label_multi[i, nbr-1] = max(1, cur_nbr)
                    


### Validation

In [38]:
print(label_binary.shape)
print(label_multi.shape)

(730, 782)
(730, 782)


In [39]:
print('In label_binary, the number of category 1 is {}'.format(np.count_nonzero(label_binary == 1)))
print('In label_multi, the number of category 1 is {}'.format(np.count_nonzero(label_multi == 1)))
print('In label_multi, the number of category 2 is {}'.format(np.count_nonzero(label_multi == 2)))

In label_binary, the number of category 1 is 10098
In label_multi, the number of category 1 is 7432
In label_multi, the number of category 2 is 2666


In [40]:
# 求在365天里所有管道的泄露天数和，对上一单元格结果进行验证，由于没有重合点所以正好是10098的一半
leakage_list = df_leakage['linkID'].to_list()
leakage_day_count = 0
abrupt_day_count = 0
for i in range(len(leakage_list)):
    linkID = leakage_list[i]
    leakage_info = df_leakage[df_leakage['linkID'] == linkID]
    start_date_str, end_date_str = leakage_info['startTime'].values[0], leakage_info['endTime'].values[0]
    start_date, end_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M'), datetime.strptime(end_date_str, '%Y-%m-%d %H:%M')
    start_date_year, end_date_year = start_date.year, end_date.year
    start_date_day, end_date_day = int(start_date.date().strftime('%j')), int(end_date.date().strftime('%j'))
    if  start_date_year == end_date_year:
        leakage_day_count += end_date_day - start_date_day + 1
        if leakage_info['leakType'].item() == 'abrupt':
            abrupt_day_count += end_date_day - start_date_day + 1
    elif end_date_year > start_date_year:
        day_ = 365*(end_date_year - start_date_year - 1) + (365 - start_date_day + 1) + end_date_day
        leakage_day_count += day_
        if leakage_info['leakType'].item() == 'abrupt':
            abrupt_day_count += day_
print('The 905 pipes\' total leakage days number is:', leakage_day_count)
print('{} of those days are abrupt leakages'.format(abrupt_day_count))

The 905 pipes' total leakage days number is: 5049
1333 of those days are abrupt leakages


## 2.5 Save Labels

In [41]:
path_ = './data/generator_data/{}_{}_{}/'.format(year_start_, year_end_, mode)
label_binary_path = path_ + 'label_binary_{}_{}.npy'.format(year_start_, year_end_)
label_multi_path = path_ + 'label_multi_{}_{}.npy'.format(year_start_, year_end_)
np.save(label_binary_path, label_binary)
np.save(label_multi_path, label_multi)

In [42]:
# load labels
label_binary = np.load(label_binary_path)
label_multi = np.load(label_multi_path)

### Draft

In [43]:
leakage_info = df_leakage[df_leakage['linkID'] == 'p579']
start_date_str, end_date_str = leakage_info['startTime'].item(), leakage_info['endTime'].item()
start_date, end_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M'), datetime.strptime(end_date_str, '%Y-%m-%d %H:%M')
start_date_year, end_date_year = start_date.year, end_date.year
start_date_day, end_date_day = int(start_date.date().strftime('%j')), int(end_date.date().strftime('%j'))
print(start_date_year, end_date_year, start_date_day, end_date_day)

ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
year_i = 2016
day_i = 150
if  (year_i == 2016 and 100 <= day_i <= 200) or year_i > 2016:
    print(True)

True


In [ ]:
df_leakage[df_leakage['linkID'] == 'p479']

,linkID,startTime,endTime,leakDiameter (m),leakType,peakTime
32,p479,2017-11-20 11:55,2017-12-31 23:55,0.013195,incipient,2017-12-31 23:55


In [ ]:
start_date_str = df_leakage[df_leakage['linkID'] == 'p479']['startTime'].values[0]
start_date_str

'2017-11-20 11:55'

In [ ]:
start_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M')
start_date

datetime.datetime(2017, 11, 20, 11, 55)

In [ ]:
start_date_year = start_date.year
start_date_year

2017

In [ ]:
start_date_day = int(start_date.date().strftime('%j'))
start_date_day

324

# 1. Generating training input with reconstructed dataset.

## 1.1 Preparation

### Importing public libraries

In [3]:
import pandas as pd
from datetime import datetime
from scipy.sparse import csr_matrix
import pickle as pkl

## 1.2 from SCADA Dataset get Pressure Dataframe

### Input: the reconstructed dataset

In [4]:
# Runtime configuration
path_to_wdn     = './data/L-TOWN.inp'
path_to_data    = './data/l-town-data/'
scaling         = 'minmax'

In [5]:
# Import the .inp file using the EPYNET library
wdn = epynet.Network(path_to_wdn)

# Solve hydraulic model for a single timestep
wdn.solve()

# Convert the file using a custom function, based on:
# https://github.com/BME-SmartLab/GraphConvWat 
G , pos , head = get_nx_graph(wdn, weight_mode='pipe_length', get_head=True)

# Instantiate the nominal WDN model
nominal_wdn_model = epanetSimulator(path_to_wdn, path_to_data)

# Run a simulation
nominal_wdn_model.simulate()

# Retrieve the nodal pressures
nominal_pressure = nominal_wdn_model.get_simulated_pressure()

In [6]:
# Run a simulation
nominal_wdn_model.simulate()

# Retrieve the nodal pressures
nominal_pressure = nominal_wdn_model.get_simulated_pressure()

In [7]:
# Open the dataset configuration file
with open(path_to_data + 'dataset_configuration.yml') as file:

    # Load the configuration to a dictionary
    config = yaml.load(file, Loader=yaml.FullLoader) 

# Generate a list of integers, indicating the number of the node
# at which a  pressure sensor is present
sensors = [int(string.replace("n", "")) for string in config['pressure_sensors']]

In [8]:
x,y,scale,bias = dataCleaner(pressure_df    = nominal_pressure, # Pass the nodal pressures
                                observed_nodes = sensors,          # Indicate which nodes have sensors
                                rescale        = scaling)          # Perform scaling on the timeseries data

In [39]:
def read_prediction(filename='predictions.csv', scale=1, bias=0, start_date='2018-01-01 00:00:00'):
    df = pd.read_csv(filename, index_col='Unnamed: 0', error_bad_lines=False)
    df.columns = ['n{}'.format(int(node)+1) for node in df.columns]
    df = df*scale+bias
    df.index = pd.date_range(start=start_date,
                             periods=len(df),
                             freq = '5min')
    return df

In [40]:
r18 = read_prediction(filename='./data/reconstruction/2018_reconstructions.csv',
                      scale=scale,
                      bias=bias,
                      start_date='2018-01-01 00:00:00')
#r19 = read_prediction(filename='2019_reconstructions.csv',
#                      scale=scale,
#                      bias=bias,
#                      start_date='2019-01-01 00:00:00')

/home/zczlx79/.conda/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  after removing the cwd from sys.path.


In [41]:
r18

,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,...,n773,n774,n775,n776,n777,n778,n779,n780,n781,n782
2018-01-01 00:00:00,28.562206,28.345151,28.691012,33.351742,36.292562,30.925312,25.692300,37.237950,32.524860,26.289917,...,52.280690,50.741119,51.865852,45.266454,48.531864,46.148552,46.436781,47.713853,49.495235,48.984960
2018-01-01 00:05:00,28.567774,28.351294,28.697684,33.359345,36.307173,30.932307,25.694896,37.253195,32.534221,26.294759,...,52.357504,50.830278,51.948323,45.366107,48.615235,46.225695,46.514532,47.791951,49.581799,49.067457
2018-01-01 00:10:00,28.572511,28.356520,28.703339,33.365064,36.319244,30.937661,25.697106,37.265770,32.541300,26.298845,...,52.347329,50.819906,51.940132,45.350956,48.601830,46.213295,46.501317,47.779018,49.566422,49.052999
2018-01-01 00:15:00,28.576546,28.360940,28.708097,33.369123,36.329063,30.941548,25.698984,37.275980,32.546340,26.302253,...,52.381570,50.858994,51.975901,45.400267,48.639778,46.248510,46.537049,47.814727,49.606479,49.091743
2018-01-01 00:20:00,28.576546,28.360940,28.708097,33.369123,36.329063,30.941548,25.698984,37.275980,32.546340,26.302253,...,52.288283,50.750411,51.874218,45.279305,48.543950,46.159290,46.447015,47.724331,49.506765,48.995606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 23:35:00,28.489195,28.271735,28.611710,33.285153,36.129260,30.862164,25.661092,37.064297,32.442085,26.233843,...,52.104959,50.530899,51.667659,45.036035,48.350939,45.979454,46.267406,47.543005,49.307805,48.803088
2018-12-31 23:40:00,28.491299,28.273457,28.613600,33.286860,36.133101,30.863721,25.661840,37.068628,32.444225,26.235182,...,52.070823,50.492146,51.632348,44.989198,48.313581,45.944307,46.231678,47.507360,49.267128,48.764091
2018-12-31 23:45:00,28.496181,28.279420,28.620028,33.293656,36.147157,30.870104,25.664290,37.082928,32.452582,26.239741,...,52.126677,50.558479,51.694150,45.062825,48.374998,46.001000,46.288533,47.564756,49.331158,48.825319
2018-12-31 23:50:00,28.496181,28.279420,28.620028,33.293656,36.147157,30.870104,25.664290,37.082928,32.452582,26.239741,...,52.087000,50.510986,51.648936,45.008603,48.329180,45.959429,46.247236,47.522781,49.286096,48.782057


### Generate graphs for training (only 2018 dataset used as training dataset)

In [20]:
begin_year = r18.index[0].date().year
end_year = r18.index[-1].date().year
graphs = []
for i in range((end_year - begin_year + 1)*365): # range(365)
    tmp_feature = []
    for node in G.nodes(): # 1-782
        tmp_feature.append(np.array(r18.iloc[288*i : 288*(i+1), node-1].tolist()))
    G.graph["feature"] = csr_matrix(tmp_feature) 
    graphs.append(G.copy())

In [24]:
len(graphs)

365

In [25]:
graphs[364].graph["feature"] # 782 * 288 = 225216

<782x288 sparse matrix of type '<class 'numpy.float64'>'
	with 225216 stored elements in Compressed Sparse Row format>

In [26]:
graphs[364][780][781] # adjs[364][779, 780]

{'weight': 0.5911166229382008, 'name': 'p901'}